In [1]:
import tensorflow as tf
import os
import json
from pathlib import Path
from skimage import io
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from scipy.sparse import csr_matrix


2024-03-30 16:11:30.901900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 16:11:30.902019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 16:11:31.221092: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
api_key = {
'username':"floreageorge" ,
'key':"c392f1958292932eb60951c0fa720f33"}

kaggle_path = Path('/root/.kaggle')
os.makedirs(kaggle_path, exist_ok=True)

with open (kaggle_path/'kaggle.json', 'w') as handl:
    json.dump(api_key,handl)

os.chmod(kaggle_path/'kaggle.json', 600)

!kaggle competitions download -c nitro-nlp-3
!unzip nitro-nlp-3.zip

 97%|████████████████████████████████████▉ | 79.0M/81.4M [00:01<00:00, 56.5MB/s]
100%|██████████████████████████████████████| 81.4M/81.4M [00:01<00:00, 50.7MB/s]
Archive:  nitro-nlp-3.zip
  inflating: random_seed_setter.py   
  inflating: sample_test_solution.csv  
  inflating: test.csv                
  inflating: train.csv               


In [3]:
train = pd.read_csv('train.csv')
test =  pd.read_csv('test.csv')

In [4]:
train['title'] = train['title'].str.lower()
train['content'] = train['content'].str.lower()
train['labels'] = train['class'].astype(int)

In [5]:
!pip install spacy
!python -m spacy download ro_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 71.3 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_sm')


In [6]:
import spacy
from spacy.lang.ro.stop_words import STOP_WORDS

nlp = spacy.load("ro_core_news_sm", disable=["parser", "ner"])

def preprocess_texts(texts):
    texts = [str(text) if pd.notnull(text) else '' for text in texts]

    docs = nlp.pipe(texts, batch_size=50)
    preprocessed_texts = []
    for doc in docs:
        preprocessed_text = ' '.join(token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.lemma_ != "-PRON-")
        preprocessed_texts.append(preprocessed_text)
    return preprocessed_texts


train['title_processed'] = preprocess_texts(train['title'].tolist())
train['content_processed'] = preprocess_texts(train['content'].tolist())


KeyboardInterrupt: 

In [ ]:
test['title'] = test['title'].str.lower()
test['content'] = test['content'].str.lower()

test['title_processed'] = preprocess_texts(test['title'].tolist())
test['content_processed'] = preprocess_texts(test['content'].tolist())

In [ ]:
train['content_processed']=train['content_processed']+train['title_processed']
test['content_processed']=test['content_processed']+test['title_processed']

In [11]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
# Prepare Documents for Word2Vec
documents = [text.split() for text in pd.concat([train['content_processed'], test['content_processed']])]

# Train Word2Vec on the entire dataset
word2vec_model = Word2Vec(sentences=documents, vector_size=100, window=5, min_count=1, workers=4)


In [12]:
# Document Vector Function
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if not doc:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word2vec_model.wv[doc], axis=0)

# Apply the Function to Create Document Vectors
train_vectors = np.array([document_vector(word2vec_model, doc.split()) for doc in train['content_processed']])
test_vectors = np.array([document_vector(word2vec_model, doc.split()) for doc in test['content_processed']])


In [13]:
import numpy as np

# Assuming train_vectors and test_vectors are already defined

# Save train_vectors to CSV
np.savetxt('train_vectors.csv', train_vectors, delimiter=',')

# Save test_vectors to CSV
np.savetxt('test_vectors.csv', test_vectors, delimiter=',')


In [11]:
import numpy as np

# Load train vectors from CSV

train_vectors = np.genfromtxt('/kaggle/input/vectors/train_vectors.csv', delimiter=',')

# Load test vectors from CSV
test_vectors = np.genfromtxt('/kaggle/input/vectors/test_vectors.csv', delimiter=',')


**Naive Bayes**

In [ ]:
# Train-Test Split (For Evaluating Model Performance)
from sklearn.metrics import accuracy_score

labels = train['class'].astype(int).values
X_train, X_val, y_train, y_val = train_test_split(train_vectors, labels, test_size=0.2, random_state=42)

# Initialize and Train the Classifier
clf = GaussianNB()
clf.fit(X_train, y_train)

# Predict on the Validation Set and Evaluate
y_val_pred = clf.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred)}")

**SVM**

In [ ]:
from sklearn.svm import SVC
import pandas as pd

# Initialize the SVC classifier
clf = SVC(kernel='linear', random_state=42)

# Train the classifier
clf.fit(train_vectors, labels)  # Assuming train_vectors and labels are defined

# Predict on the Test Set
test_predictions = clf.predict(test_vectors)

# Ensure test_predictions are in the format of integers 0 and 1
test_predictions_int = [int(pred) for pred in test_predictions]

# Create a DataFrame with IDs and predictions
submission_df = pd.DataFrame({
    'id': test['id'],  # Replace 'id' with the actual name of your ID column if it's different
    'class': test_predictions_int
})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission_svm.csv', index=False)

print("Submission CSV file for SVM predictions has been created.")

**CNN**

In [53]:
X_train = np.expand_dims(train_vectors, axis=2)
X_test = np.expand_dims(test_vectors, axis=2)
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout

model = Sequential()
model.add(Conv1D(filters=256, kernel_size=7, activation='relu', input_shape=(100, 1)))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
X = train_vectors  # Feature vectors
y = train['labels'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
history = model.fit(X_train, y_train, epochs=5, validation_split=0.1, batch_size=32)

from sklearn.metrics import balanced_accuracy_score

# Assuming you have predictions for your validation set
y_pred = model.predict(X_val)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

balanced_acc = balanced_accuracy_score(y_val, y_pred_binary)
print("Balanced Accuracy:", balanced_acc)

(70575, 100) (70575,)
Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1588/1588 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.8749 - loss: 0.2967 - val_accuracy: 0.9245 - val_loss: 0.1940
Epoch 2/5
1588/1588 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.9226 - loss: 0.1988 - val_accuracy: 0.9283 - val_loss: 0.1904
Epoch 3/5
1588/1588 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.9310 - loss: 0.1778 - val_accuracy: 0.9143 - val_loss: 0.2178
Epoch 4/5
1588/1588 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.9340 - loss: 0.1737 - val_accuracy: 0.9299 - val_loss: 0.1863
Epoch 5/5
1588/1588 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9347 - loss: 0.1667 - val_accuracy: 0.9394 - val_loss: 0.1645
442/442 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Balanced Accuracy: 0.9349836666230912


In [55]:
history = model.fit(X,y, epochs=5, validation_split=0.1, batch_size=32)

Epoch 1/5
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9348 - loss: 0.1714 - val_accuracy: 0.9442 - val_loss: 0.1499
Epoch 2/5
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9348 - loss: 0.1685 - val_accuracy: 0.9433 - val_loss: 0.1454
Epoch 3/5
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9372 - loss: 0.1630 - val_accuracy: 0.9432 - val_loss: 0.1445
Epoch 4/5
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9392 - loss: 0.1587 - val_accuracy: 0.9436 - val_loss: 0.1462
Epoch 5/5
1985/1985 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9389 - loss: 0.1580 - val_accuracy: 0.9432 - val_loss: 0.1474


In [77]:
y_pred = model.predict(test_vectors)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

1146/1146 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step


In [79]:
import csv
submission_df = pd.DataFrame({'class': y_pred_binary.flatten()})
print(submission_df)
submission_df.to_csv('predictionFolly.csv', index=True)
print("Submission CSV file for SVM predictions has been created.")

       class
0          0
1          1
2          1
3          1
4          0
...      ...
36664      0
36665      0
36666      1
36667      1
36668      0

[36669 rows x 1 columns]
Submission CSV file for SVM predictions has been created.


**Random Forest **

In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_vectors, train['labels'], test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
predictions = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Additional evaluation metrics
print(classification_report(y_test, predictions))


Accuracy: 0.9540913921360255
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      5053
           1       0.96      0.96      0.96      9062

    accuracy                           0.95     14115
   macro avg       0.95      0.95      0.95     14115
weighted avg       0.95      0.95      0.95     14115



In [81]:
rf_classifier.fit(X,y)
predictions = rf_classifier.predict(test_vectors)


In [83]:
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions
import csv
submission_df = pd.DataFrame({'class': predictions.flatten()})
print(submission_df)
submission_df.to_csv('predictionFolly.csv', index=True)
print("Submission CSV file for SVM predictions has been created.")

       class
0          0
1          1
2          0
3          1
4          0
...      ...
36664      0
36665      0
36666      1
36667      1
36668      0

[36669 rows x 1 columns]
Submission CSV file for SVM predictions has been created.


**Transformers**

In [84]:
!pip install transformers


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [12]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')
def bert_encode(texts, tokenizer, model, max_length=512):
    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    
    for text in texts:
        encoded_text = tokenizer.encode_plus(text, 
                                             max_length=max_length, 
                                             pad_to_max_length=True, 
                                             add_special_tokens=True, 
                                             return_attention_mask=True, 
                                             return_token_type_ids=True, 
                                             truncation=True)
        
        all_input_ids.append(encoded_text['input_ids'])
        all_attention_masks.append(encoded_text['attention_mask'])
        all_token_type_ids.append(encoded_text['token_type_ids'])
    
    all_input_ids = np.array(all_input_ids)
    all_attention_masks = np.array(all_attention_masks)
    all_token_type_ids = np.array(all_token_type_ids)
    
    with torch.no_grad():
        outputs = model(input_ids=torch.tensor(all_input_ids), 
                        attention_mask=torch.tensor(all_attention_masks), 
                        token_type_ids=torch.tensor(all_token_type_ids))
    
    return outputs.last_hidden_state[:, 0, :].numpy()



train_embeddings = bert_encode(train['content_processed'].tolist(), tokenizer, model)
test_embeddings = bert_encode(test['content_processed'].tolist(), tokenizer, model)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

KeyError: 'content_processed'

In [ ]:
# Example: Train a classifier (e.g., RandomForest)
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
clf.fit(train_embeddings, train['labels'])

# Predictions
predictions = clf.predict(test_embeddings)

# Evaluation
# Make sure to have your test labels to evaluate predictions
